In [1]:
import os
import pandas as pd
import numpy as np
import random
import gc
from datetime import datetime
from tqdm import tqdm
import matplotlib.pyplot as plt
from gensim.corpora import WikiCorpus
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from  collections import Counter

np.random.seed(2019)
random.seed(2019)
pd.set_option('display.max_rows', 6)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 280)
pd.set_option('display.max_colwidth', 150)
data_path = '/data/workspace/kimi/tencent_ads/2020/dataset'
preprocess_path = 'preprocess'

In [2]:
total_merged= 'total_merged.pkl'
total_merged_path = f'{preprocess_path}/{total_merged}'
total_merged_df = pd.read_pickle(total_merged_path)
print(total_merged_df)


          time  user_id  creative_id  click_times    ad_id  product_id  product_category  advertiser_id  industry
32216896     1  3969503       146454            1   131508      -34137                18          34137        40
25196907     1     2267       249047            1   223979        1927                 2          15784       322
13936212     1   512898       168451            1   150988        1243                 2          29966       322
...        ...      ...          ...          ...      ...         ...               ...            ...       ...
58813490    91  3773900      3680751            1  3167517       42718                17          10259        -1
23168403    91   498277      3877769            1  3331468       26858                 3             23        60
13125865    91   141441      4329985            1  3712996       39503                17          13328        -1

[63668283 rows x 9 columns]


In [5]:

def deepwalk(log,f1,f2,flag,L):
    #Deepwalk算法，
    print("deepwalk:",f1,f2)
    #构建图
    dic={}
    for item in tqdm(log[[f1,f2]].values,total=len(log)):
        try:
            str(int(item[1]))
            str(int(item[0]))
        except:
            continue
        try:
            dic['item_'+str(int(item[1]))].add('user_'+str(int(item[0])))
        except:
            dic['item_'+str(int(item[1]))]=set(['user_'+str(int(item[0]))])
        try:
            dic['user_'+str(int(item[0]))].add('item_'+str(int(item[1])))
        except:
            dic['user_'+str(int(item[0]))]=set(['item_'+str(int(item[1]))])
    dic_cont={}
    for key in dic:
        dic[key]=list(dic[key])
        dic_cont[key]=len(dic[key])
    print("creating")     
    #构建路径
    path_length=10        
    sentences=[]
    length=[]
    for key in tqdm(dic,total=len(dic)):
        sentence=[key]
        while len(sentence)!=path_length:
            key=dic[sentence[-1]][random.randint(0,dic_cont[sentence[-1]]-1)]
            if len(sentence)>=2 and key == sentence[-2]:
                break
            else:
                sentence.append(key)
        sentences.append(sentence)
        length.append(len(sentence))
        if len(sentences)%100000==0:
            print(len(sentences))
    print(np.mean(length))
    print(len(sentences))
    #训练Deepwalk模型
    print('training...')
    random.shuffle(sentences)
    model = Word2Vec(sentences, size=L, window=4,min_count=1,sg=1, workers=10,iter=20)
    print('outputing...')
    model.save(f'deepwalk_{f2}_{L}')
    #输出
    values=set(log[f1].values)
    w2v=[]
    for v in values:
        try:
            a=[int(v)]
            a.extend(model['user_'+str(int(v))])
            w2v.append(a)
        except:
            pass
    out_df=pd.DataFrame(w2v)
    names=[f1]
    for i in range(L):
        names.append(f1+'_'+ f2+'_'+names[0]+'_deepwalk_embedding_'+str(L)+'_'+str(i))
    out_df.columns = names
    print(out_df.head())
    out_df.to_pickle('preprocess_path/' +f1+'_'+ f2+'_'+f1 +'_'+flag +'_deepwalk_'+str(L)+'.pkl') 
    ########################
    values=set(log[f2].values)
    w2v=[]
    for v in values:
        try:
            a=[int(v)]
            a.extend(model['item_'+str(int(v))])
            w2v.append(a)
        except:
            pass
    out_df=pd.DataFrame(w2v)
    names=[f2]
    for i in range(L):
        names.append(f1+'_'+ f2+'_'+names[0]+'_deepwalk_embedding_'+str(L)+'_'+str(i))
    out_df.columns = names
    print(out_df.head())
    out_df.to_pickle('preprocess_path/' +f1+'_'+ f2+'_'+f2 +'_'+flag +'_deepwalk_'+str(L)+'.pkl') 

In [ ]:
deepwalk(total_merged_df,'user_id','advertiser_id','train',64)

deepwalk: user_id advertiser_id



  5%|▍         | 2916974/63668283 [00:21<04:23, 230711.52it/s]


  9%|▉         | 5839944/63668283 [00:38<04:49, 199942.63it/s]


 14%|█▍        | 8823938/63668283 [00:51<03:49, 238558.00it/s]


 18%|█▊        | 11745085/63668283 [01:03<03:38, 237646.68it/s]


 23%|██▎       | 14659951/63668283 [01:15<03:26, 237274.21it/s]


 28%|██▊       | 17542862/63668283 [01:28<03:16, 234270.97it/s]


 32%|███▏      | 20074002/63668283 [01:39<03:07, 232454.67it/s]